In [ ]:
import pandas as pd
import numpy as np
import json

LABEL_FILE_PATH = './label_train_filtered.csv'

In [ ]:
label_df = pd.read_csv(LABEL_FILE_PATH)
#label_df.info    

In [ ]:
role_list = [2,3,5,6,7,9,12,17,19,21,22]
#role_list = [2]
new_id = 0
new_data = []
xyz_count = 0
total_xyz_count = 0
convert_dict = {
    '7':0,
    '3':1,
    '20':2,
    '5':3,
    '6':4,
    '24':5,
    '16':6,
    '17':7,
    '18':8,
    '26':9
}

# try all user's data file
for r in role_list:
    xyz_count = 0
    act_counter = [0 for i in range(10)]
    ACC_USER_PATH = './data/acc_user_filter' + str(r) + '.csv'
    acc_df = pd.read_csv(ACC_USER_PATH)
    acc_df = acc_df.sort_values(by='datetime')              # sort the datetime
    #print(acc_df.info)
    print(r)
    timestamp_list = acc_df['datetime'].tolist()
    #print(len(timestamp_list))
    

    def binary_search(left, right, target):
    
        while left <= right:
            middle = (left + right) // 2

            if timestamp_list[middle] < target:
                left = middle + 1
            elif timestamp_list[middle] > target:
                right = middle - 1
            else:
                return middle
        return left

    user_act = label_df[label_df['user_id'] == r]           # find corresponding user's activities
    #print(user_act.info)
    print("user ", r, "'s activity number", len(user_act))

    max_timestamp, min_timestamp = acc_df['datetime'].max(), acc_df['datetime'].min()
    
    # filter out raws with start time greater than finish time
    user_act = user_act[user_act['start'] <= user_act['finish']]
    start_time, end_time = user_act['start'].tolist(), user_act['finish'].tolist()
    cur_label = user_act['activity_type_id']
    
    # this is remove duplicated elements
    #filtered_act = list(set(zip(start_time, end_time)))

    filtered_act = list(zip(start_time,end_time,cur_label))

    print("after filtered: ", len(filtered_act))

    #print(len(start_time), len(end_time))

    for s, e, act_label in filtered_act:
        #print(act_label)
        if s <= e and s > min_timestamp and s < max_timestamp:
            left, right = 0, len(timestamp_list)-1
            begining = binary_search(left, right, s)
            xyz_list = []
            ending = begining
            #print(ending, right)
            while ending <len(timestamp_list) and  timestamp_list[ending] <= e:
                ending += 1

            #if begining >= ending:
            #    print("start time: ", s, " finish time", e)
                
            #print("activity: ", act_label, " has match? ", begining < ending)
            
            if begining < ending:
                selected = acc_df[begining:ending]
                x,y,z = selected['x'].tolist(), selected['y'].tolist(), selected['z'].tolist()
                timestamp = selected['datetime'].tolist()
                #print(selected.info)

                xyz = []
                xyz_filter = set()
                for index in range(len(x)):
                    if timestamp[index] not in xyz_filter:
                        elm = [x[index],y[index],z[index]]
                        xyz.append(elm)
                        xyz_filter.add(timestamp[index])
                    
                
                meta = {
                    'id': new_id,
                    'user': r,
                    'label': convert_dict[str(act_label)],
                    'ole_label': int(act_label),
                    'start': s,
                    'finish': e,
                    'acc': xyz
                }
                new_id += 1
                
                act_counter[convert_dict[str(act_label)]] += 1
                total_xyz_count += len(xyz)
                xyz_count += len(xyz)
                #print(new_id, len(x), begining, ending)
                new_data.append(meta)
            
    print(act_counter)
    print("current activities:", new_id, ". Increased xyz data count ", xyz_count, " total xyz data:", total_xyz_count)


2
user  2 's activity number 744
after filtered:  636
[142, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current activities: 142 . Increased xyz data count  1573  total xyz data: 6878064
3
user  3 's activity number 66
after filtered:  63
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
current activities: 142 . Increased xyz data count  1573  total xyz data: 6878064
5
user  5 's activity number 141
after filtered:  125
[0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
current activities: 144 . Increased xyz data count  15678  total xyz data: 6964809
6
user  6 's activity number 262
after filtered:  262
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
current activities: 146 . Increased xyz data count  40  total xyz data: 6972901


KeyboardInterrupt: 

In [ ]:
f = open('data.json', 'w')
f.write(json.dumps(new_data))
f.close()